### Clean version of Pommeroye catchment

In [1]:
import os
import pandas as pd

# Instal geemap package
import subprocess
import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

The Pommeroy catchment is at 50°28'19.1"N and 2°03'40.5"E, or in decimal:

In [2]:
lat = 50 + (28/60) + 0.01*(19.1/60)
lon = 2 + (3/60) + 0.01*(40.5/60)
print('lat: {}°N, lon: {}°E'.format(lat, lon))

lat: 50.46985°N, lon: 2.0567499999999996°E


In [3]:
# Get Sentinel 2 collections, both Level-1C (top of atmophere) and Level-2A (surface reflectance)
MSILCcol = ee.ImageCollection('COPERNICUS/S2')
MSI2Acol = ee.ImageCollection('COPERNICUS/S2_SR')

In [4]:
# Define a collection filter by date, bounds, and quality.
def colFilter(col, roi, start_date, end_date):
    return(col
           .filterBounds(roi)
           .filterDate(start_date, end_date))

 # Function to get and rename bands of interest from MSI
def renameMSI(img):
    return(img.select(
        ['B2', 'B3', 'B4', 'B5', 'B6', 'B7',
            'B8', 'B8A', 'B11', 'B12', 'QA60'],
        ['Blue', 'Green', 'Red', 'Red Edge 1', 'Red Edge 2', 'Red Edge 3', 'NIR', 'Red Edge 4',
            'SWIR1', 'SWIR2', 'QA60']))

# Add NBR for LandTrendr segmentation.
def calcNbr(img):
    return(img.addBands(img.normalizedDifference(['NIR', 'SWIR2'])
                        .multiply(-10000).rename('NBR')).int16())

# Define function to mask out clouds and cloud shadows in images.
# Use CFmask band included in USGS Landsat SR image product.
def fmask(img):
    cloudOpaqueBitMask = 1 << 10
    cloudCirrusBitMask = 1 << 11
    qa = img.select('QA60')
    mask = qa.bitwiseAnd(cloudOpaqueBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudCirrusBitMask).eq(0))
    return(img.updateMask(mask))

# Define function to prepare MSI images.
def prepMSI(img):
    orig = img
    img = renameMSI(img)
    img = fmask(img)
    return(ee.Image(img.copyProperties(orig, orig.propertyNames()))
           .resample('bicubic'))

In [5]:
# Make a dummy image for missing years.
bandNames = ee.List(['Blue', 'Green', 'Red', 'Red Edge 1',
                     'Red Edge 2', 'Red Edge 3', 'NIR',
                     'Red Edge 4', 'SWIR1', 'SWIR2', 'QA60'])
fillerValues = ee.List.repeat(0, bandNames.size())
dummyImg = ee.Image.constant(fillerValues).rename(bandNames) \
    .selfMask().int16()

In [6]:
months = geemap.date_sequence('2016-01-01', '2020-01-01', 'month')
roi=ee.Geometry.Rectangle([lon-0.05, lat-0.025, lon+0.05, lat+0.025])

In [7]:
# Get monthly median collection.
def getMonthlyComp(m):
    #print(m)
    startDate = ee.Date(m)
    endDate = startDate.advance(ee.Number(4), 'month')  # I find that a 4 monthly average gets rid of clouds...

    # Filter collections and prepare them for merging.
    MSILCcoly = colFilter(MSILCcol, roi, startDate, endDate).map(prepMSI)
    MSI2Acoly = colFilter(MSI2Acol, roi, startDate, endDate).map(prepMSI)

    # Merge the collections.
    col = MSILCcoly.merge(MSI2Acoly)

    yearImg = col.median()
    print('yearImg: {}'.format(type(yearImg)))
    nBands = yearImg.bandNames().size()
    yearImg = ee.Image(ee.Algorithms.If(
        nBands,
        yearImg,
        dummyImg))
    print('yearImg at end: {}'.format(type(yearImg)))
    return(calcNbr(yearImg)
           .set({'year': m, 'system:time_start': startDate.millis(), 'nBands': nBands}))

In [8]:
imgList = months.map(getMonthlyComp)

yearImg: <class 'ee.image.Image'>
yearImg at end: <class 'ee.image.Image'>
yearImg: <class 'ee.image.Image'>
yearImg at end: <class 'ee.image.Image'>


In [9]:
# Convert image composite list to collection
imgCol = ee.ImageCollection.fromImages(imgList)
imgCol = imgCol.map(lambda img: img.clip(roi))

In [13]:
# Define arguments for animation function parameters.
videoArgs = {
  'dimensions': 768,
  'region': roi,
  'framesPerSecond': 0.1,
  'crs': 'EPSG:3857',
  'bands': ['Blue', 'Green', 'Red'],
  'min': 0,
  'max': 5000,
  'gamma': 1.3
}

In [14]:
saved_gif = os.path.join(os.path.expanduser('~'), 'Downloads/pommeroye.gif')
geemap.download_ee_video(imgCol, videoArgs, saved_gif)

Generating URL...
Please wait ...
The GIF image has been saved to: /work/armitagj/Downloads/pommeroye.gif


In [15]:
text = pd.date_range('1/1/2016','1/31/2020',freq='MS').strftime('%Y-%m-%d')
out_gif = os.path.join(os.path.expanduser('~'), 'Downloads/pommeroye2.gif')
geemap.add_text_to_gif(saved_gif, out_gif, xy=('3%', '5%'), text_sequence=text, font_size=30, font_color='#ffffff')
geemap.add_text_to_gif(out_gif, out_gif, xy=('32%', '92%'), text_sequence='Pommeroye Catchment', font_color='white')

In [16]:
!convert ~/Downloads/pommeroye2.gif -fuzz 23.5% -layers Optimize ~/Downloads/pommeroye_compressed.gif